# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.235237817225                   -0.50    7.0         
  2   -7.249811893791       -1.84       -1.39    1.0   11.7ms
  3   -7.251150196008       -2.87       -1.97    2.0   13.6ms
  4   -7.251236009059       -4.07       -2.11    2.0   14.4ms
  5   -7.251296471986       -4.22       -2.34    1.0   13.4ms
  6   -7.251337017100       -4.39       -3.01    1.0   13.7ms
  7   -7.251338657678       -5.79       -3.43    2.0   18.2ms
  8   -7.251338781438       -6.91       -3.90    1.0   81.3ms
  9   -7.251338795328       -7.86       -4.28    2.0   14.7ms
 10   -7.251338798419       -8.51       -4.85    1.0   14.7ms
 11   -7.251338798657       -9.62       -5.17    3.0   16.7ms
 12   -7.251338798699      -10.38       -5.76    2.0   22.6ms
 13   -7.251338798704      -11.32       -6.15    2.0   18.9ms
 14   -7.251338798704      -12.19       -6.45    3.0   17.3ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.051546166983921726
[ Info: Arnoldi iteration step 2: normres = 0.4557021687482518
[ Info: Arnoldi iteration step 3: normres = 0.7893499779827291
[ Info: Arnoldi iteration step 4: normres = 0.48454808335939176
[ Info: Arnoldi iteration step 5: normres = 0.47652728226183794
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.01e-01, 7.83e-02, 3.75e-01, 2.64e-01, 1.42e-02)
[ Info: Arnoldi iteration step 6: normres = 0.29821511343177404
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.59e-02, 1.28e-01, 2.34e-01, 9.28e-02, 8.99e-02)
[ Info: Arnoldi iteration step 7: normres = 0.07346004348908544
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (8.97e-04, 9.73e-03, 1.27e-02, 2.53e-02, 5.53e-02)
[ Info: Arnoldi iteration step 8: normres = 0.09314319500237628
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (3.7